# llama2-chat으로 generate
## 해당 증상에 해당하는 코로나 환자의 발화 생성하기 

### 모델, 토크나이저

In [1]:
import os
import numpy as np
from torch import nn
import re
from collections import defaultdict
import torch
from torch.nn.utils.rnn import pad_sequence
from datasets import Dataset, DatasetDict
from src.data import CDialogueDataset
import pandas as pd
import torch, transformers
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from transformers import LlamaForCausalLM, LlamaTokenizer,TextGenerationPipeline


<frozen importlib._bootstrap>:219: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /home/wooseok/miniconda3/envs/mh/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/wooseok/miniconda3/envs/mh/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda113_nocublaslt.so...


/home/wooseok/miniconda3/envs/mh/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


In [2]:
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model =LlamaForCausalLM.from_pretrained(model_name,
                                        load_in_8bit=True,
                                        device_map="auto" # gpu 꽉차면 cpu로 올려줌 
                                       )

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### data

In [3]:
#dataset = CDialogueDataset.load_dataset(tokenizer, max_length=256)

In [4]:
from datasets import load_from_disk 
dataset = load_from_disk('/home/wooseok/llama2-chat/CDialog-clean/')

In [5]:
conversation = dataset['test']['utterances'][2]
conversation

['Patient: Hello Doctor, On Friday, I was diagnosed with pneumonia and the chest x-ray states Minimal linear subsegmental atelectasis in the left lower lobe.',
 'Doctor: Are you taking any Antibiotics?',
 'Patient: I have been taking an antibiotic since Friday. I am still feeling tired and drained.',
 'Doctor: Tiredness is expected for 4-5 days at least.',
 'Patient: Is this to be expected with minimal linear subsegmental atelectasis in the left lower lobe?',
 "Doctor: Minimal linear atelectasis doesn't mean pneumonia of the lung, but still if you are diagnosed as pneumonia then it's ok.",
 'Patient: Thanks a lot Doctor.',
 'Doctor: Hope this clears your query. Welcome for any further assistance. Regards.']

In [6]:
#a = dataset['test']['symptoms'][2]
#a

In [7]:
a = dataset['test']['attributes'][2]
a

[['pneumonia'], [], [], [], [], [], [], []]

### 생성 함수 정의 


#### history가 있을 때
- 다음 대화에 이어질 환자의 발화를 (cough 증상에 대해 말하도록) 생성해줘. 

In [8]:
def generate_text_with_history(prompt, max_length=512):
    text_generation_pipeline = TextGenerationPipeline(
        model = model,
        tokenizer = tokenizer,
        torch_dtype =  torch.float16
    )
    generated_sequences = text_generation_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=max_length,
    )
    recommendations = generated_sequences[0]["generated_text"].replace(prompt,'')
    if " Doctor:" in recommendations : 
        recommendations = recommendations[:recommendations.find(" Doctor:")]
    if " Doctore:" in recommendations : 
        recommendations = recommendations[:recommendations.find(" Doctore:")]
    if "\nDoctor:" in recommendations : 
        recommendations = recommendations[:recommendations.find("\nDoctor:")]
    if "\nDoctore:" in recommendations : 
        recommendations = recommendations[:recommendations.find("\nDoctore:")]
    #print(recommendations)

    return recommendations


result = []
for i in range(len(dataset['test'])):
#for i in range(1,3):
    historys = dataset['test']['utterances'][i]
    print(" ")
    print(i,"번째 데이터")
    
    Total_generated_sentence = []
    for k in range(2,len(dataset['test']['attributes'][i]),2):
        Symptoms = dataset['test']['attributes'][i][k]
        Symptoms = ', '.join(Symptoms)
        #if Symptoms[-1] =='.':
        #    Symptoms = Symptoms[:-1]
        if Symptoms == 'nan':
            Symptoms = ''
        print("환자의",k,"번째 발화에 대한 증상 :" ,Symptoms)
        
        history=''
        for p in range(k):
            history = history+ " "+historys[p]
        input_prompt="Generate suspected coronavirus patient's utterance about %s symptoms to continue the next conversation.%s Patient:"% (Symptoms ,history)
        #print("Prompt:",input_prompt )
        
        num_samples = 1
        list_of_generated_sentence= []
        if (i == 50 and k > 5) :
            generated_sentence = '' 
            list_of_generated_sentence.append(generated_sentence)

        else:
            print("------------------------------------------------------------------------------------------------------------------------")
            generated_sentence = generate_text_with_history(input_prompt)
            list_of_generated_sentence.append(generated_sentence)
            print(generated_sentence)
        Total_generated_sentence.append(list_of_generated_sentence)
    result.append(Total_generated_sentence)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


 
0 번째 데이터
환자의 2 번째 발화에 대한 증상 : 
------------------------------------------------------------------------------------------------------------------------
 I have a throat problem and headache. What should I do?. A throat infection is usually not life-threatening, but it can cause a person a lot of discomfort and can also make them vulnerable to other infections. A throat infection can be caused by a bacterial infection, a viral infection or an allergic reaction. A person may also get a throat infection if their immune system is weakened for any. The throat area is very sensitive to touch. Avoid rubbing your throat, even to alleviate itchiness. If you have a sore throat, try to avoid eating and drinking as much as possible, or use an ice pack to alleviate throat pain. You can use an over-the-counter medicine to soothe your throat. If you have a fever, take aspirin or acetaminophen. Throat infection, also called tonsillitis, is a medical condition of the tonsils (glandular structures in 

Input length of input_ids is 528, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


 hi, i had sex with a girl 15 yrs. old and the sex lasted for one minute and she had a condom.i was afraid that i may get hiv because we were both naked.please help me sir.is it possible that she could transmit the hiv to me by any other ways except by the blood.i am very worried about it. please reply to this as soon as possible.
환자의 4 번째 발화에 대한 증상 : 
------------------------------------------------------------------------------------------------------------------------


Input length of input_ids is 635, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


...

 doctor
환자의 10 번째 발화에 대한 증상 : 
------------------------------------------------------------------------------------------------------------------------


Input length of input_ids is 765, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


 thanks
환자의 12 번째 발화에 대한 증상 : 
------------------------------------------------------------------------------------------------------------------------
 okay
 
49 번째 데이터
환자의 2 번째 발화에 대한 증상 : 
------------------------------------------------------------------------------------------------------------------------
 I do not have fever.
환자의 4 번째 발화에 대한 증상 : 
------------------------------------------------------------------------------------------------------------------------
 Yes, doctor.  Patient: I am also worried.
 
50 번째 데이터
환자의 2 번째 발화에 대한 증상 : 
------------------------------------------------------------------------------------------------------------------------
 no i dont know what should i do?
환자의 4 번째 발화에 대한 증상 : 
------------------------------------------------------------------------------------------------------------------------
 Thank u sir for your answer i want to ask some question. Sir i have seen some patient they have problem in tongue so i want to know that i have pr

Input length of input_ids is 595, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


 Thank
환자의 6 번째 발화에 대한 증상 : 
------------------------------------------------------------------------------------------------------------------------
 Thanks
 
91 번째 데이터
환자의 2 번째 발화에 대한 증상 : 
------------------------------------------------------------------------------------------------------------------------
 I want to know the cost of operation for my leg.
환자의 4 번째 발화에 대한 증상 : pneumonia
------------------------------------------------------------------------------------------------------------------------
 A sudden chill in a warm environment.
환자의 6 번째 발화에 대한 증상 : 
------------------------------------------------------------------------------------------------------------------------
 It's a month.
환자의 8 번째 발화에 대한 증상 : 
------------------------------------------------------------------------------------------------------------------------
 I was running down the stairs at home. Suddenly, I had difficulty breathing and felt like I was dying.
환자의 10 번째 발화에 대한 증상 : 
------------------

In [9]:
print(len(result))

print(len(result[1]))

97
4


In [10]:
print(len(result))
print(len(result[1]))


97
4


In [11]:
import pickle


with open("llama_2_execpt_first_3.pkl","wb") as f:
    pickle.dump(result, f)